In [ ]:
import sys
import mysql.connector
import xlsxwriter
from xlsxwriter.utility import xl_rowcol_to_cell
import pandas as pd
import sys
import mysql.connector
import os
import datetime
from dateutil.relativedelta import relativedelta
import numpy as np

year = datetime.date.today().year

quincena=4
# quincena=2
month = np.ceil(quincena/ 2)
isFirstHalf = quincena % 2 != 0
startDate =  str(year)+"-"+str(int(month)).zfill(2)+"-01" if isFirstHalf else  str(year)+"-"+str(int(month)).zfill(2)+"-16"
endDate =  str(year)+"-"+str(int(month)).zfill(2)+"-15" if isFirstHalf else  str((datetime.datetime(year,int(month),1 )+relativedelta(months=1))-datetime.timedelta(days=1))[:10];

from dotenv import load_dotenv
load_dotenv()
#configurar la conexion a la base de datos
DB_USERNAME = os.getenv('DB_USERNAME')
DB_DATABASE = os.getenv('DB_DATABASE')
DB_PASSWORD = os.getenv('DB_PASSWORD')
DB_PORT = os.getenv('DB_PORT')
a_color='#354F84'
b_color='#91959E'
# Conectar a DB
cnx = mysql.connector.connect(user=DB_USERNAME,
                              password=DB_PASSWORD,
                              host='localhost',
                              port=DB_PORT,
                              database=DB_DATABASE,
                              use_pure=False)
#Seccion para traer informacion de la base
query = ('SELECT * from customers where id = 1')

# join para cobros
# cobros=pd.read_sql('Select cobros.* ,customers.customer,internal_orders.invoice, users.name from ((cobros inner join internal_orders on internal_orders.id = cobros.order_id) inner join customers on customers.id = internal_orders.customer_id )inner join users on cobros.capturo=users.id',cnx)
#Calcular el intervalo segun num de quincena


#traer datos de los pedidos
programados=pd.read_sql('select * from payments',cnx)
clientes=pd.read_sql("""select  * from customers """,cnx)
bancos=pd.read_sql("""select  * from banks """,cnx)

pedidos=pd.read_sql("""Select internal_orders.* ,customers.clave,customers.alias,
coins.exchange_sell, coins.coin, coins.symbol, coins.code,sellers.seller_name
from (((
    internal_orders
    inner join customers on customers.id = internal_orders.customer_id )
    inner join coins on internal_orders.coin_id = coins.id)
    inner join  sellers on sellers.id=internal_orders.seller_id)
     """,cnx)

cobros=pd.read_sql("""select cobro_orders.*,cobros.comp,cobros.date,cobros.bank_id,
                   internal_orders.customer_id,internal_orders.invoice,internal_orders.noha,
                   internal_orders.seller_id,internal_orders.comision,internal_orders.total
                     from (((
                         cobro_orders 
    inner join cobros on cobros.id=cobro_orders.cobro_id)
    inner join internal_orders on internal_orders.id = cobros.order_id )
    inner join coins on internal_orders.coin_id = coins.id)
                   where cobros.date >= '"""+startDate+"' and cobros.date <= '"+endDate+"'",cnx)

facturas=pd.read_sql("""select factures.*,cobro_factures.cobro_id
                     from (((
                         factures
    inner join internal_orders on internal_orders.id = factures.order_id )
    inner join cobro_factures on cobro_factures.facture_id=factures.id )
    inner join coins on internal_orders.coin_id = coins.id) """,cnx)

creditos=pd.read_sql("""select * 
                     from ((
                         credit_notes    inner join internal_orders on internal_orders.id = credit_notes.order_id )
    inner join coins on internal_orders.coin_id = coins.id) """,cnx)

vendedores=pd.read_sql("""select * 
                     from sellers where status='ACTIVO'""",cnx)

socios=pd.read_sql("select * from sellers where  dgi > 0",cnx)

no_socios=pd.read_sql("select * from sellers where status='ACTIVO' and dgi <= 0",cnx)
comisiones=pd.read_sql("""select * 
                     from comissions""",cnx)

nordenes=len(pedidos)
df=pedidos[['date']]

tc=pd.read_sql('select * from coins where id=13 ',cnx)['exchange_sell'].values[0]
writer = pd.ExcelWriter('storage/report/dgi_resumen_ejecutivos'+str(quincena-1)+'.xlsx', engine='xlsxwriter')
workbook = writer.book
##FORMATOS PARA EL TITULO------------------------------------------------------------------------------
rojo_l = workbook.add_format({
    'bold': 0,
    'border': 0,
    'align': 'center',
    'valign': 'vcenter',
    #'fg_color': 'yellow',
    'font_color': 'red',
    'font_size':16})
negro_s = workbook.add_format({
    'bold': 0,
    'border': 0,
    'align': 'center',
    'valign': 'vcenter',
    'font_color': 'black',
    'font_size':11})
firmas = workbook.add_format({
    'bold': 0,
    'top': 1,
    'align': 'center',
    'valign': 'vcenter',
    'font_color': 'black',
    'font_size':12})

negro_b = workbook.add_format({
    'bold': 2,
    'border': 0,
    'align': 'center',
    'valign': 'vcenter',
    'font_color': 'black',
    'font_size':13,
    
    'text_wrap': True,
    'num_format': 'dd/mm/yyyy'}) 

rojo_b = workbook.add_format({
    'bold': 2,
    'border': 0,
    'align': 'center',
    'valign': 'vcenter',
    'font_color': 'red',
    'font_size':13})      
divisor = workbook.add_format({
    'bold': True,
    'text_wrap': True,
    'valign': 'center',
    'bg_color': '#696e78',
    'border': 0,})

#FORMATOS PARA CABECERAS DE TABLA --------------------------------
header_format = workbook.add_format({
    'bold': True,
    'text_wrap': True,
    'valign': 'center',
    'fg_color': 'yellow',
    'border': 1,})

blue_header_format = workbook.add_format({
    'bold': True,
    'bg_color': a_color,
     'text_wrap': True,
    'valign': 'vcenter',
    'align': 'center',
    'border_color':'white',
    'font_color': 'white',
    'border': 1})
blue_header_format_bold = workbook.add_format({
    'bold': True,
    'bg_color': a_color,
    'text_wrap': True,
    'valign': 'vcenter',
    'align': 'center',
    'border_color':'white',
    'font_color': 'white',
    'border': 1,
    'num_format': '[$$-409]#,##0.00',
    'font_size':13})

blue_footer_format_bold = workbook.add_format({
    'bold': True,
    'bg_color': a_color,
    'text_wrap': True,
    'valign': 'top',
    'align': 'center',
    'border_color':'white',
    'font_color': 'white',
    'border': 1,
    'num_format': '[$$-409]#,##0.00',
    'font_size':11})
#FORMATOS PARA TABLAS PER CE------------------------------------

blue_content = workbook.add_format({
    'border': 1,
    'align': 'center',
    'valign': 'vcenter',
    'font_color': 'black',
    
    'border_color':a_color,
    'font_size':10,
    'num_format': '[$$-409]#,##0.00'})
blue_content_red = workbook.add_format({
    'border': 1,
    'align': 'center',
    'valign': 'vcenter',
    'font_color': 'red',
    
    'border_color':a_color,
    'font_size':10,
    'num_format': '[$$-409]#,##0.00'})
blue_content_unit = workbook.add_format({
    'border': 1,
    'align': 'center',
    'valign': 'vcenter',
    'font_color': 'black',
    
    'border_color':a_color,
    'font_size':10,
    })


blue_content_dll = workbook.add_format({
    'border': 1,
    'align': 'center',
    'valign': 'vcenter',
    'font_color': 'black',
    'bg_color': '#b4e3b1',
    'border_color':a_color,
    'font_size':10,
    'num_format': '[$$-409]#,##0.00'})

blue_content_bold = workbook.add_format({
    'bold': True,
    'border': 1,
    'align': 'center',
    'valign': 'vcenter',
    'font_color': 'black',
    'font_size':11,
    'border_color':a_color,
    'num_format': '[$$-409]#,##0.00'})

blue_content_bold_dll = workbook.add_format({
    'bold': True,
    'border': 1,
    'align': 'center',
    'valign': 'vcenter',
    'font_color': 'black',
    'font_size':11,
    'bg_color': '#b4e3b1',
    'border_color':a_color,
    'num_format': '[$$-409]#,##0.00'})
blue_content_footer_dll = workbook.add_format({
    'bold': True,
    'border': 1,
    'align': 'center',
    'valign': 'vcenter',
    'font_color': 'white',
    'font_size':11,
    'bg_color': '#356e31',
    'border_color':'white',
    'num_format': '[$$-409]#,##0.00'})
blue_content_footer = workbook.add_format({
    'bold': True,
    'border': 1,
    'align': 'center',
    'valign': 'vcenter',
    'font_color': 'white',
    'font_size':11,
    'bg_color': '#3e5585',
    'border_color':'white',
    'num_format': '[$$-409]#,##0.00'})
blue_content_date = workbook.add_format({
    'border': 1,
    'align': 'center',
    'valign': 'vcenter',
    'font_color': 'black',
    'font_size':9,
    'border_color':a_color,
    'num_format': 'dd/mm/yyyy'})
#FOOTER FORMATS---------------------------------------------------------
observaciones_format = workbook.add_format({
    'bold': True,
    'text_wrap': True,
    'valign': 'top',
    'fg_color':'#BDD7EE',
    'border': 1})

total_cereza_format = workbook.add_format({
    'bold': True,
    'text_wrap': True,
    'valign': 'top',
    'fg_color':'#F4B084',
    'border': 1})


import datetime

currentDateTime = datetime.datetime.now()
date = currentDateTime.date()
year = date.strftime("%Y")
#Columna para filtrar por fechas
pedidos['date']=pd.to_datetime(pedidos['date'])

# -------------HOJA DE RESUMEN
worksheet= workbook.add_worksheet("Resumen")
#Encabezado del documento--------------
worksheet.write('G2', 'AÑO', negro_b)
worksheet.write('H2', year, negro_b)
worksheet.merge_range('G2:H3', """FECHA DEL REPORTE
DD/MM/AAAA""", negro_b)
worksheet.merge_range('I2:I3', date, negro_b)
worksheet.insert_image("A1", "img/logo/logo.png",{"x_scale": 0.6, "y_scale": 0.6})
worksheet.merge_range('B3:F4', """TABLA DE VENDEDORES PARA PAGO DE COMISIONES  
                      DGI PARA NIVELES DIRECTIVOS""", negro_b)
worksheet.merge_range('B5:F5', "Se reporta del "+str(startDate) +" al "+ str(endDate), negro_s)

##SEGUNDA TABLA DE RESUMEN VENTAS DIRECTAS
#Cabeceras
worksheet.merge_range(5,1,8,1,"""SIN IVA
comision generada por el monto""",blue_header_format)
worksheet.merge_range(5,2,8,2,"""
PEDIDO INTERNO""",blue_header_format)
worksheet.write(5,3,'No. vendedor',blue_header_format)
worksheet.write(6,3,'Iniciales',blue_header_format)
worksheet.write(7,3,'Nombre corto',blue_header_format)
worksheet.write(8,3,'comp.Ingesos',blue_header_format)
#total de cada xobro columna
for i in range(len(cobros)):
    this_comisions=comisiones.loc[comisiones['order_id']==cobros['order_id'].values[i]]
    worksheet.write(9+i,1,(cobros['amount'].values[i]/1.16)*this_comisions['percentage'].sum(),blue_content)
    worksheet.write(9+i,2,cobros['invoice'].values[i],blue_content)
    worksheet.write(9+i,3,cobros['comp'].values[i],blue_content)
for i in range(len(socios)):
    this_comisions=comisiones.loc[comisiones['seller_id']==socios['id'].values[i]]
    worksheet.write(5,4+i,str(i+1),blue_header_format)
    worksheet.write(6,4+i,socios['iniciales'].values[i],blue_header_format)
    worksheet.write(7,4+i,str(socios['seller_name'].values[i]).split()[-1],blue_header_format)
    worksheet.write(8,4+i,'comision $',blue_header_format)
    worksheet.write(len(cobros)+10, 4+i,str(socios['seller_name'].values[i]).split()[-1],blue_content_footer)
    for j in range(len(cobros)):
        comision_secundaria=this_comisions.loc[this_comisions['order_id']==cobros['order_id'].values[j]]
        amount=0
        if(cobros['seller_id'].values[j]==socios['id'].values[i]):
           amount=(cobros['amount'].values[j])*cobros['comision'].values[j]
        
        if(len(comision_secundaria)>0):
           amount=(cobros['amount'].values[j])*comision_secundaria['percentage'].values[0]
        worksheet.write(9+j,4+i,amount,blue_content)

for i in range(len(socios)):
        # Definir el rango para la fórmula
    start_cell = xl_rowcol_to_cell(9, 4+i)  # Primera celda (0, 0 -> A1)
    end_cell = xl_rowcol_to_cell(len(cobros) +8, 4+i) 
    # Crear la fórmula SUM para sumar la columna
    formula = f"=SUM({start_cell}:{end_cell})"

    # Escribir la fórmula en una celda (por ejemplo, en la fila 6, columna 1 -> A6)
    worksheet.write_formula(len(cobros)+9, 4+i, formula,blue_footer_format_bold)

worksheet.write_formula(len(cobros)+9, 1, f"=SUM(B10:B"+str(len(cobros)+9)+")",blue_footer_format_bold)
worksheet.set_row(5,27)

#AGRANDAR CPLUMNAS
worksheet.set_column('A:A',15)
worksheet.set_column('B:B',20)
worksheet.set_column('F:F',25)
worksheet.set_column('G:G',35)
worksheet.set_column('E:O',18)
worksheet.set_column('P:T',15)
worksheet.set_column('U:AX',19)
worksheet.set_landscape()
worksheet.set_paper(9)
worksheet.fit_to_pages(1, 1)  
workbook.close()


13793.103448275862 6.832350000000001
107000.00000000001 6.832350000000001
56000.00000000001 6.832350000000001
3950.4482758620697 6.832350000000001
16725.0 6.832350000000001
1603038.5 6.832350000000001
54450.00000000001 6.832350000000001
1008000.0000000001 6.832350000000001
45850.0 6.832350000000001
43240.00000000001 6.832350000000001
37670.0 6.832350000000001
3265.5172413793107 6.832350000000001
127300.00000000001 6.832350000000001
81650.0 6.832350000000001
16300.000000000002 6.832350000000001
24856.439655172417 6.832350000000001
35000.0 6.832350000000001
83619.0 6.832350000000001
7545.206896551726 6.832350000000001
6337.931034482759 6.832350000000001


c:\Users\fgonzalez\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
c:\Users\fgonzalez\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
c:\Users\fgonzalez\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
c:\Users\fgonzalez\AppData\Local\Programs\Python\Python310\lib\site

In [4]:
comisiones

,id,order_id,seller_id,percentage,dgi,created_at,updated_at,description
0,1,104,5,0.080000,1.0,2023-03-26,2023-03-26,None
1,2,92,1,0.010000,0.0,2023-03-26,2023-03-26,tyrsa
2,3,92,5,0.000000,0.0,2023-03-26,2023-03-26,kl
3,4,92,4,0.070000,1.0,2023-03-26,2023-03-26,DGI
4,5,93,9,0.000100,0.0,2023-03-27,2023-03-27,TYRSA
...,...,...,...,...,...,...,...,...
1935,1952,711,28,0.000100,0.0,2025-03-27,2025-03-27,DGI
1936,1953,709,2,0.013932,0.0,2025-03-27,2025-03-28,DGI DV
1937,1954,709,4,0.006956,0.0,2025-03-27,2025-03-28,DGI GAF
1938,1955,709,5,0.006956,0.0,2025-03-27,2025-03-28,DGI GVT


In [20]:
orders.loc[orders['date'].astype(str)>str(date.strftime("%Y"))+'-01-01',"total"]

423    5.183228e+04
424    1.003168e+05
425    1.026716e+04
426    1.295720e+04
427    1.088080e+05
428    1.128912e+05
429    6.275600e+04
430    7.946000e+04
431    6.739600e+04
432    4.609944e+03
433    8.120000e+03
434    3.961400e+05
435    5.707200e+04
436    2.587960e+05
437    8.651280e+03
438    1.063720e+05
439    2.489360e+03
440    1.651724e+06
441    2.683672e+05
442    3.209720e+05
443    3.209720e+05
444    3.430120e+05
445    1.894280e+05
446    6.275832e+04
447    0.000000e+00
448    1.940100e+04
449    4.369720e+04
450    1.263240e+05
451    0.000000e+00
452    2.487040e+05
453    9.326400e+04
454    2.379160e+06
455    0.000000e+00
456    4.717720e+06
457    2.379160e+06
458    4.609944e+03
459    6.496000e+04
460    1.476680e+05
461    0.000000e+00
462    3.781600e+04
463    0.000000e+00
464    0.000000e+00
465    5.707200e+04
466    0.000000e+00
Name: total, dtype: float64

In [16]:
orders.loc[orders['date'].astype(str)>'2025-01-01','total'].sum()

14925724.8488

In [19]:
str(date.strftime("%Y"))+'01-01'

'202501-01'

In [ ]:
import datetime
from dateutil.relativedelta import relativedelta
import numpy as np
year = datetime.date.today().year
quincena=12
month = np.ceil(quincena/ 2)
isFirstHalf = quincena % 2 != 0
startDate =  str(year)+"-"+str(int(month)).zfill(2)+"-01" if isFirstHalf else  str(year)+"-"+str(int(month)).zfill(2)+"-15"
endDate =  str(year)+"-"+str(int(month)).zfill(2)+"-14" if isFirstHalf else  str((datetime.datetime(year,int(month),1 )+relativedelta(months=1))-datetime.timedelta(days=1))[:10];

2025-06-15 2025-06-30


In [ ]:
#siguiente mes en dia primero


'2025-05-31'

In [11]:
'1'.zfill(2)

'01'

In [19]:
facturas.loc[facturas['id'].isin(cobros_facturas.loc[cobros_facturas['cobro_id'].isin(cobros['cobro_id'].values)].facture_id.values)]

,id,order_id,ordinal,facture,amount,status,npagos,created_at,updated_at,date,coin_id,tc
1,826,620,1,A 4115,818738.0,CAPTURA,None,2025-01-10,2025-01-10,2025-01-10,1,1.0
2,835,620,1,A 4125,1309980.0,CAPTURA,None,2025-01-27,2025-01-27,2025-01-21,1,1.0


In [20]:
facturas

,id,order_id,ordinal,facture,amount,status,npagos,created_at,updated_at,date,coin_id,tc
0,821,620,1,A 4110,818738.0,CAPTURA,None,2025-01-03,2025-01-03,2024-12-30,1,1.0
1,826,620,1,A 4115,818738.0,CAPTURA,None,2025-01-10,2025-01-10,2025-01-10,1,1.0
2,835,620,1,A 4125,1309980.0,CAPTURA,None,2025-01-27,2025-01-27,2025-01-21,1,1.0


In [23]:
818738.0+1309980.0

2128718.0

In [42]:
for i in cobros.cobro_id.unique():
    print(i)
    fac_ids=cobros_facturas.loc[cobros_facturas['cobro_id']==i].facture_id.unique()
    for j in fac_ids:
        print('',j)
        print(len(facturas.loc[facturas['id']==j,['id','facture']]))

214
 150
1
 159
1
215
 151
1
216
 152
1
 153
1
217
 154
1
 155
1
249
 157
1
 158
1
 212
1
483
 417
1
 426
0
 440
1
 441
1
 442
0
487
 374
1
 443
1
542
 455
1
573
 515
1
635
 515
1
665
 544
1


In [10]:
cobros.facture_id

AttributeError: 'DataFrame' object has no attribute 'facture_id'

In [32]:
all_facturas

NameError: name 'all_facturas' is not defined

In [18]:
cobros_facturas.loc[cobros_facturas['cobro_id']==679]

,cobro_id,facture_id,id,created_at,updated_at
639,679,167,1034,2025-01-26,2025-01-26


In [26]:
allcobros['ordinal']=range(1,len(allcobros)+1)

C:\Users\hp\AppData\Local\Temp\ipykernel_14000\141872390.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  allcobros['ordinal']=range(1,len(allcobros)+1)


In [27]:
allcobros

,cobro_id,order_id,id,created_at,updated_at,amount,comp,date,bank_id,customer_id,invoice,noha,seller_id,comision,total,ordinal
33,58,155,40,None,None,1252626.0,1862,2022-12-29,2,446,2950,2022,14,0.003,2505252.0,1
34,59,155,41,None,None,1002100.8,1940,2023-03-28,2,446,2950,2022,14,0.003,2505252.0,2
540,595,155,580,2024-03-01,2024-03-01,250525.2,2297,2024-03-01,2,446,2950,2022,14,0.003,2505252.0,3


In [ ]:
cobros[['invoive']]

,cobro_id,order_id,id,created_at,updated_at,amount,comp,date,bank_id,customer_id,invoice,noha,seller_id,comision,total
0,21,116,4,None,None,150000.00,1066,2020-02-24,2,506,2175,12020,2,0.030,356120.00
1,22,116,5,None,None,20000.00,1472,2021-08-16,2,506,2175,12020,2,0.030,356120.00
2,23,116,6,None,None,20000.00,1492,2021-09-17,2,506,2175,12020,2,0.030,356120.00
3,24,116,7,None,None,23200.00,1526,2021-11-12,2,506,2175,12020,2,0.030,356120.00
4,25,116,8,None,None,23200.00,1552,2021-12-23,2,506,2175,12020,2,0.030,356120.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
632,695,332,688,2024-05-17,2024-05-17,55482.68,2381,2024-05-16,1,418,3102,1412023,14,0.003,1090632.00
633,695,451,689,2024-05-17,2024-05-17,92624.84,2381,2024-05-16,1,418,3102,1412023,14,0.003,1090632.00
634,696,438,690,2024-05-17,2024-05-17,444048.00,2382,2024-05-16,2,337,3191,42,5,0.003,1110120.00
635,697,398,691,2024-05-17,2024-05-17,502194.16,2383,2024-05-17,2,424,3160,11,2,0.003,1062388.32
